In [1]:
import pandas as pd
import numpy as np
import lightgbm
import catboost
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import mutual_info_regression

In [2]:
train1 = pd.read_csv("datasets/median_data_train.csv")
test1 = pd.read_csv("datasets/median_data_test.csv")

In [3]:
test1_dates = test1["tarih"]
train1.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)
test1.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)

In [4]:
test1.head()

,ilce,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,izmir-aliaga,0,38.7996,26.9707,14.15,24.85,0.0,83.10,167.85,4.25,1.0,14.50
1,izmir-aliaga,0,38.7996,26.9707,15.20,9.05,0.0,79.85,159.10,1.65,1.0,15.30
2,izmir-aliaga,0,38.7996,26.9707,14.85,52.05,0.0,69.75,133.75,4.00,1.0,14.50
3,izmir-aliaga,0,38.7996,26.9707,14.95,81.60,0.0,78.45,175.60,8.05,1.0,14.35
4,izmir-aliaga,0,38.7996,26.9707,14.85,52.35,0.0,68.45,187.80,6.90,1.0,13.60


In [5]:
train1.head()

,ilce,bildirimsiz_sum,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,izmir-aliaga,5.0,0.0,38.7996,26.9707,12.85,50.75,0.0,88.00,170.00,4.75,1.0,13.00
1,izmir-aliaga,3.0,0.0,38.7996,26.9707,12.45,36.40,0.0,83.90,106.85,3.00,1.0,12.60
2,izmir-aliaga,2.0,0.0,38.7996,26.9707,12.10,57.30,0.0,78.90,120.80,3.45,1.0,12.00
3,izmir-aliaga,0.0,0.0,38.7996,26.9707,14.15,48.05,0.0,67.85,113.95,4.55,1.0,14.05
4,izmir-aliaga,8.0,0.0,38.7996,26.9707,14.20,22.65,0.0,79.20,178.85,3.95,1.0,14.20


In [6]:
ord_coder = OrdinalEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()

In [7]:
X = train1.drop("bildirimsiz_sum",axis=1)
y = train1["bildirimsiz_sum"]

In [8]:
dummies = ohe.fit_transform(X[["ilce"]])
dummies = dummies.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummy = pd.DataFrame(data=dummies,columns=features)

In [9]:
X = X.drop("ilce",axis=1)

In [10]:
X = pd.concat([X,dummy],axis=1)

In [11]:
X

,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,...,ilce_manisa-koprubasi,ilce_manisa-kula,ilce_manisa-salihli,ilce_manisa-sarigol,ilce_manisa-saruhanli,ilce_manisa-sehzadeler,ilce_manisa-selendi,ilce_manisa-soma,ilce_manisa-turgutlu,ilce_manisa-yunusemre
0,0.0,38.7996,26.9707,12.85,50.75,0.0,88.00,170.00,4.75,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,38.7996,26.9707,12.45,36.40,0.0,83.90,106.85,3.00,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,38.7996,26.9707,12.10,57.30,0.0,78.90,120.80,3.45,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,38.7996,26.9707,14.15,48.05,0.0,67.85,113.95,4.55,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,38.7996,26.9707,14.20,22.65,0.0,79.20,178.85,3.95,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51601,0.0,39.1881,27.6100,9.75,70.15,0.0,86.80,225.30,1.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51602,1.0,39.1881,27.6100,11.75,24.60,0.0,74.65,153.05,2.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51603,0.0,39.1881,27.6100,12.50,70.05,0.0,71.65,152.70,3.80,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51604,1.0,39.1881,27.6100,14.00,66.35,0.0,76.75,155.70,2.90,1.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
X["ilce"] = ord_coder.fit_transform(X[["ilce"]])

In [11]:
minmax = MinMaxScaler()

In [12]:
cols = dummy.columns.to_list()

In [13]:
cols.append("bildirimli_sum")

In [14]:
mm_columns = X.drop(cols,axis=1).columns

In [10]:
mm_columns = X.drop(["ilce","bildirimli_sum"],axis=1).columns

In [15]:
minmax.fit(X[mm_columns])
X[mm_columns] = minmax.fit_transform(X[mm_columns])

In [12]:
X

,ilce,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,0.0,0.0,0.68601,0.260122,0.484581,0.5075,0.0,0.858989,0.473528,0.309028,0.000000,0.534884
1,0.0,0.0,0.68601,0.260122,0.475771,0.3640,0.0,0.810811,0.295214,0.187500,0.000000,0.526795
2,0.0,0.0,0.68601,0.260122,0.468062,0.5730,0.0,0.752056,0.334604,0.218750,0.000000,0.514661
3,0.0,0.0,0.68601,0.260122,0.513216,0.4805,0.0,0.622209,0.315262,0.295139,0.000000,0.556117
4,0.0,0.0,0.68601,0.260122,0.514317,0.2265,0.0,0.755582,0.498518,0.253472,0.000000,0.559151
...,...,...,...,...,...,...,...,...,...,...,...,...
51601,46.0,0.0,1.00000,0.509255,0.416300,0.7015,0.0,0.844888,0.629677,0.083333,0.000000,0.467139
51602,46.0,1.0,1.00000,0.509255,0.460352,0.2460,0.0,0.702115,0.425667,0.152778,0.000000,0.514661
51603,46.0,0.0,1.00000,0.509255,0.476872,0.7005,0.0,0.666863,0.424679,0.243056,0.000000,0.520728
51604,46.0,1.0,1.00000,0.509255,0.509912,0.6635,0.0,0.726792,0.433150,0.180556,0.010235,0.556117


In [16]:
dummies = ohe.transform(test1[["ilce"]])
dummies = dummies.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummy = pd.DataFrame(data=dummies,columns=features)
test1 = test1.drop("ilce",axis=1)
test1 = pd.concat([test1,dummy],axis=1)
cols = dummy.columns.to_list()
cols.append("bildirimli_sum")
mm_columns = test1.drop(cols,axis=1).columns
test1[mm_columns] = minmax.transform(test1[mm_columns])

In [18]:
test1["ilce"] = ord_coder.transform(test1[["ilce"]])
mm_columns = test1.drop(["ilce","bildirimli_sum"],axis=1).columns
test1[mm_columns] = minmax.transform(test1[mm_columns])

KeyError: "['ilce'] not found in axis"

In [14]:
regressor = catboost.CatBoostRegressor(
    n_estimators=200,
    learning_rate=0.1,
    depth=12,
    l2_leaf_reg=5,loss_function="MAE"
)
# regressor = SVR(kernel="poly",degree=2,C=1.5)

In [25]:
mi = mutual_info_regression(X, y)
feature_importance = np.argsort(mi)[::-1]
feature_importance

array([ 0,  2,  3,  1, 11,  7,  5, 10,  9,  8,  6,  4], dtype=int64)

In [26]:
new_x = X.copy()
new_x = new_x.iloc[:,feature_importance[:8]]

In [27]:
new_x

,ilce,lat,lon,bildirimli_sum,t_apparent:C,relative_humidity_2m:p,effective_cloud_cover:p,prob_precip_1h:p
0,0.0,0.68601,0.260122,0.0,0.534884,0.858989,0.5075,0.000000
1,0.0,0.68601,0.260122,0.0,0.526795,0.810811,0.3640,0.000000
2,0.0,0.68601,0.260122,0.0,0.514661,0.752056,0.5730,0.000000
3,0.0,0.68601,0.260122,0.0,0.556117,0.622209,0.4805,0.000000
4,0.0,0.68601,0.260122,0.0,0.559151,0.755582,0.2265,0.000000
...,...,...,...,...,...,...,...,...
51601,46.0,1.00000,0.509255,0.0,0.467139,0.844888,0.7015,0.000000
51602,46.0,1.00000,0.509255,1.0,0.514661,0.702115,0.2460,0.000000
51603,46.0,1.00000,0.509255,0.0,0.520728,0.666863,0.7005,0.000000
51604,46.0,1.00000,0.509255,1.0,0.556117,0.726792,0.6635,0.010235


In [28]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
maes = []
predictions = []
i = 0
for train_index, test_index in skf.split(X, y):
    print(f"{i+1}.Fold")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    maes.append(mean_absolute_error(y_test,y_pred))
    predictions.append(regressor.predict(test1))
    i+=1

c:\Users\LOKMAN\anaconda3\envs\MlEnv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


1.Fold
0:	learn: 3.5544893	total: 108ms	remaining: 21.4s
1:	learn: 3.4757594	total: 210ms	remaining: 20.8s
2:	learn: 3.4084065	total: 323ms	remaining: 21.2s
3:	learn: 3.3505445	total: 423ms	remaining: 20.7s
4:	learn: 3.2911314	total: 529ms	remaining: 20.6s
5:	learn: 3.2357143	total: 633ms	remaining: 20.5s
6:	learn: 3.1916263	total: 736ms	remaining: 20.3s
7:	learn: 3.1608539	total: 863ms	remaining: 20.7s
8:	learn: 3.1254445	total: 974ms	remaining: 20.7s
9:	learn: 3.0975854	total: 1.1s	remaining: 20.9s
10:	learn: 3.0714143	total: 1.24s	remaining: 21.2s
11:	learn: 3.0506728	total: 1.37s	remaining: 21.4s
12:	learn: 3.0343426	total: 1.49s	remaining: 21.5s
13:	learn: 3.0156550	total: 1.61s	remaining: 21.4s
14:	learn: 2.9990747	total: 1.74s	remaining: 21.5s
15:	learn: 2.9768403	total: 1.88s	remaining: 21.7s
16:	learn: 2.9621437	total: 2.09s	remaining: 22.6s
17:	learn: 2.9460532	total: 2.23s	remaining: 22.5s
18:	learn: 2.9312610	total: 2.35s	remaining: 22.4s
19:	learn: 2.9231102	total: 2.47s	r

In [29]:
maes

[2.7658515245061226,
 2.7445634246836708,
 2.6912114861596184,
 2.7116862016455956,
 2.737504022579036,
 2.7404299419262133,
 2.738129593593143,
 2.726054205602232,
 2.7388051858874607,
 2.7309461565319357]

In [29]:
predictions

[array([4.5877466 , 4.0210182 , 4.07615158, ..., 4.81546415, 4.53656969,
        5.53807344]),
 array([4.53681709, 4.35708621, 3.98140589, ..., 4.34934624, 4.5584981 ,
        4.42374069]),
 array([4.74515691, 4.38611164, 4.11894493, ..., 4.68972775, 5.17457126,
        5.22198518]),
 array([4.57223026, 4.31616917, 4.36899156, ..., 5.5365198 , 6.02386852,
        6.52431657]),
 array([4.32262291, 4.02529656, 4.10909997, ..., 4.7905505 , 5.51926576,
        6.06409497]),
 array([4.19181082, 4.15356352, 4.10878157, ..., 4.08194702, 3.73777077,
        4.4260112 ]),
 array([4.36501392, 3.91708724, 4.20665206, ..., 4.90166099, 5.63074113,
        4.86877248]),
 array([4.43964866, 4.18150709, 4.00389554, ..., 5.16933097, 4.70321583,
        5.95629001]),
 array([4.5139495 , 4.50311286, 4.01845842, ..., 4.48781382, 4.51672122,
        5.50424903]),
 array([4.68197375, 4.42170719, 3.89396758, ..., 4.45344469, 4.77653171,
        4.98373034])]

In [30]:
predicts = np.mean(predictions,axis=0)

In [33]:
inverse_ilceler = ohe.inverse_transform(test1[dummy.columns])

In [34]:
test1 = test1.drop(dummy.columns,axis=1)

In [35]:
inverse_ilceler = inverse_ilceler.reshape((1316,))

In [36]:
test1["ilce"] = inverse_ilceler
test1["tarih"] = test1_dates

In [37]:
test1["tarih"] = test1["tarih"].astype("str")

In [38]:
unique_id = test1["tarih"] + "-" + test1["ilce"]

In [40]:
submission = pd.DataFrame({"unique_id":unique_id,"bildirimsiz_sum":predicts})

In [41]:
submission.to_csv("catboost.csv",index=False)

In [105]:
submission.to_csv("submission_SVR.csv",index=False)

In [107]:
import joblib
joblib.dump(regressor,"regressor_ensemble.joblib")

['regressor_ensemble.joblib']

In [42]:
a = pd.read_csv("submission2.csv")
mean_absolute_error(a["bildirimsiz_sum"],predicts)

0.8849766001400747